<div class="cite2c-biblio"></div><img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> INF-493 - Machine Learning</h1>
    <h1> Tarea 1 - Metodos Lineales para Regresión </h1>
</center>
<p>
<br><center>_Javier Reyes_<strong> - </strong>_javier.reyes.12@sansano.usm.cl_<strong> - </strong>_201273524-6_ </center>
<br><center>_Marco Salinas_<strong> - </strong>_marco.salinas.12@sansano.usm.cl_<strong> - </strong>_201273589-0_ </center>
</p>

# Introducción


#  1 Regresión Lineal Ordinaria (LSS)


<p  style="text-align: justify;"> </p>

<p  style="text-align: justify;"> 
    **(a)** Construya un dataframe con los datos a analizar descargándolos desde la plataforma como se indicó.
Explique por qué se realiza la línea 4.
</p>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("kc_house_data.csv")
df.drop(['id','date','zipcode',],axis=1,inplace=True)
df.head()

<p  style="text-align: justify;">
    La función _drop_ utilizada en el código anterior, elimina las columnas "id", "date" y "zipcode" del dataframe, que anteriormente habiamos leido. Esto se realiza debido a que dichos atributos no son relevantes para el estudio del dataframe.
</p>

<p  style="text-align: justify;"> 
    **(b)** Describa brevemente el dataset a utilizar.
</p>

In [ ]:
print(df.shape)
print("------------------------------------------")
df.info()
df.describe()

<p  style="text-align: justify;">
    El dataset posee $21613$ datos, los cuales poseen $18$ atributos cada uno donde describen las distintas caracteristicas de las casa, por ejemplo, el precio de venta de la casa, la cantidad de habitaciones y baños, la cantidad de metros cuadrados construidos, y los del terreno. Además, de otras especificaciones tecnicas como el año de construcción, el año de la ultima renovacion, el promedio de metros cuadrados construidos y de terreno de las 15 casas más cercanas. 
</p>

<p  style="text-align: justify;">
    Todos estos datos nos ayudarian a tomar decisiones sobre cual casa se deberia comprar.
</p>

<p  style="text-align: justify;">
    **(c)** Normalice los datos antes de trabajar y aplique una transformación adecuada a la variable a predecir.
Explique la importancia/conveniencia de realizar estas dos operaciones.
</p>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['price'] = np.log(df['price'])

<p  style="text-align: justify;">
    **(d)** Realice una regresión lineal de mínimos cuadrados básica. Explique la importancia/conveniencia del
paso 4 y los argumentos que se deben entregar a la función que implementa la regresión lineal.
</p>

In [ ]:
import sklearn.linear_model as lm
X = df_scaled.iloc[:,1:] #use .ix instead, in older pandas version
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['price']
#mascara estatica con el 70% de los datos
mascara = np.zeros(len(X))
limit = int(len(X)*0.7)
mascara[:limit] = 1
istrain = mascara== 1
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

<p  style="text-align: justify;">
    **(e)** Construya una tabla con los pesos y Z-score correspondientes a cada predictor (variable). ¿Qué variables
están más correlacionadas con la respuesta? Si usáramos un nivel de significación del 5%. ¿Qué es lo
que observa y cuál puede ser la causa?
</p>


In [ ]:
import numpy as np
import scipy.stats as sc
N = Xtrain.shape[0]
p = Xtrain.shape[1]
X = Xtrain.as_matrix() #datos pasados a matriz
invXTX = np.linalg.inv(np.mat(X.T)*np.mat(X)) #matriz inversa de X.T*X 
b_hat_left = np.mat(invXTX)*np.mat(X.T) #matriz inversa de X.T*X por X.T
b_hat = np.squeeze(np.asarray(b_hat_left.dot(ytrain)))# inversa(X.T*X)*X.T*y
yhat_train = linreg.predict(Xtrain) # y gorro desde predicción regresión lineal aplicada a los datos X de entrenamiento
aux_sig = np.mean(np.power(yhat_train - ytrain, 2))#estimador de varianza 
cte = 1/(N-p-1) #N datos con p inputs
Naux_sig = N*aux_sig
est_sig = cte*Naux_sig#estimador de sigma
v_j = np.diag(invXTX)
wz = np.empty((18,2))
for i in range(0,18):
    wz[i][0] = b_hat[i]
    wz[i][1] = b_hat[i]/(np.power(est_sig*v_j[i],0.5))
df = pd.DataFrame(wz, columns = ["Peso","Z-score"])
df.index = ["bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition",
"grade","sqft_above","sqft_basement","yr_built","yr_renovated","lat","long","sqft_living15","sqft_lot15","intercept"] 
t_value = stats.t.ppf(1-0.05, N-p-1)
print("Intervalo de confianza con un 5%% de significancia aplicando t_student = [%f,%f]"%(-1*t_value,t_value))
df


<p  style="text-align: justify;">
    Las variables más correlocionadas son: $grade = 0.233575$, $sqft-living15 = 0.085933$ y $floors = 0.08539$
    donde grade es la variable con más correlación con la respuesta.<br>
    Los <b><i>Z-Score</i></b> calculados se obtienen de la siguiente manera:
   <br><br><center>
   $\begin{eqnarray}
   z_j = \frac{\hat{\beta_j}}{\hat{\sigma} \sqrt{v_j}}
   \end{eqnarray}$
   </center><br><br>
   Donde:
   <center>
   $\begin{eqnarray}
      \hat{\beta_j} &=& (X^TX)^{-1}X^Ty \\
      v_J &=& Diag(X^TX)^{-1})\\
      \hat{\sigma^2} &=& \frac{1}{N-p-1}\sum_{i=1}^{N}(y_i - \hat{y_i})^2 
   \end{eqnarray}$
   </center><br><br>
    El comportamiento de los <b><i>Z-Score</i></b> está dominado una distribución <b><i>T-Student</i></b> con $N-p-1 = 15110$ grados de libertad, donde el intervalo de confianza que se obtiene para estos datos de entrenamiento y sus predicciones con grado de significancia del <b>5%</b>  es $[-1.644954,1.644954]$. Con esto se puede observar que hay 4 atributos que tiene valor absoluto menor a $1.644954$, los cuales son:<br><br>
    <li>$sqft-living = |8.031012e-07|$</li>
    <li>$sqft-above = |6.555770e-06|$</li>
    <li>$sqft-basement = |1.065057e-05|$</li>
    <li>$long = |1.58967e+00|$</li>
    <br>
    $\therefore$ Estos 4 atributos no son significantes para elegir qué casa comprar. 
</p>


<p  style="text-align: justify;">
    **(f)** Proponga un método para corregir lo observado (Hint: inspírese en los métodos de feature engineering
de las siguiente secciones). Verifíquelo mediante los Z-score presentados en la pregunta *(e)*.
</p>

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(linreg, Xtrain, ytrain,scoring="neg_mean_squared_error", cv=5)
lam = min(scores)#el mínimo cross validation para penalizar en los Ridge values
U, s, V = np.linalg.svd(X, full_matrices=False)#representación SVD de X
D = np.diag(s)
ridge_num = D.dot(U.T).dot(ytrain)
for i in range(0,D.shape[0]):
    ridge_num[i] = ridge_num[i]/(np.power(D[i][i],2)+lam)
for i in range(0,18):
    wz[i][0] = ridge_num[i]
    wz[i][1] = ridge_num[i]/(np.power(est_sig*v_j[i],0.5))
df = pd.DataFrame(wz, columns = ["Bridge Peso","Bridge Z-score"])
df.index = ["bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition",
"grade","sqft_above","sqft_basement","yr_built","yr_renovated","lat","long","sqft_living15","sqft_lot15","intercept"] 
df

<p  style="text-align: justify;">
    Para poder corregir la significancia de los Z-Score de los 4 atributos anteriormente nombrados, se utiliza la técnica de regresión <b><i>Ridge</i></b>, en donde se calculan los <b><i>Cross Validation</i></b> posibles y se escojé el mínimo para penalizar los pesos.<br><br>
    Primero se utiliza la descomposición $SVD$ de la matrix $X$, quedando la siguiente igualdad:
    <br><br><center>
    $\begin{eqnarray}
        X\hat{\beta}^{Ridge}_j &=& X(X^TX)^{-1}X^Ty\\
                               &=& UD(D^2+\lambda I)^{-1}DU^Ty\\
                               &=& \sum_{j=1}^{p}u_j\frac{d_j^2}{d^2_j + \lambda}u^Ty
     \end{eqnarray}$
    <br><br></center>
    Notar que al descomponer la matrix $X$ en su forma de $SVD$, se anula $U$ y $V$ debido a su ortonormalidad $U^TU=I$.<br>
    Una vez obtenido el $\lambda$, se reemplaza $\hat{\beta_j}$ por $\hat{\beta}^{Ridge}_j$,los cuales se calculan de la siguiente manera:
   <br><br><center>
   $\begin{eqnarray}
       \hat{\beta}^{Ridge}_j = \sum_{j=1}^{p}\frac{d_j}{d^2_j + \lambda}u^T_jy
   \end{eqnarray}$
   </center><br><br>
      Reemplazando los nuevos valores del estimador, se puede ver un cambio en los atributos en un orden de $10^5$. Aún así, no fue posible modificar la significancia de todos los atributos anteriormente descartados, el único atributo que alcanzó un valor absoluto mayor a $1.644954$ fue $long$.
</p>

<p  style="text-align: justify;">
    **(g)** Estime el error de predicción del modelo usando validación cruzada con un número de folds igual a $K
= 5$ y $K = 10$. Recuerde que para que la estimación sea razonable, en cada configuración (_fold_) debería
reajustar los pesos del modelo. Mida el error real del modelo sobre el conjunto de pruebas, compare y
concluya.
</p>

In [ ]:
yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))
print('El error de predición real usando el test_set es %f'%mse_test)
from sklearn import cross_validation
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()

k_fold = cross_validation.KFold(len(Xm),5)
mse_cv = 0
for k, (train, val) in enumerate(k_fold):
    linreg = lm.LinearRegression(fit_intercept = False)
    linreg.fit(Xm[train], ym[train])
    yhat_val = linreg.predict(Xm[val])
    mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
    mse_cv += mse_fold
mse_cv = mse_cv / 5
print('El error de prediccion usando CV con K=5 es %f'%mse_cv)

k_fold = cross_validation.KFold(len(Xm),10)
mse_cv = 0
for k, (train, val) in enumerate(k_fold):
    linreg = lm.LinearRegression(fit_intercept = False)
    linreg.fit(Xm[train], ym[train])
    yhat_val = linreg.predict(Xm[val])
    mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
    mse_cv += mse_fold
mse_cv = mse_cv / 10
print('El error de prediccion usando CV con K=10 es %f'%mse_cv)

<p  style="text-align: justify;">
    Si se utiliza $K=5$, en cada iteración habrá un $80\%$ de los datos para entrenar y $20\%$ para validar, en contraste con el 90% de datos de entranimiento y 10% de datos de validación con $K=10$. Al utilizar mayor cantidad de datos de entrenamiento habrá menor error, pero a costa del <b><i>overfitting</i></b> (sobreajuste al entrenamiento) que puede causar el entrenamiento con una cantidad considerable de datos.<br>
Al apreciar el error real respecto a los de predición de cross-validation, se observa que los resultados obtenidos son lo suficientemente buenos como para concluir qué casas comprar dependiendo de los atributos dados.
</p>

<p  style="text-align: justify;">
    **(h)** Mida los errores de predicción para cada dato de entrenamiento. Utilizando un "quantile-quantile plot"
determine si es razonable la hipótesis de normalidad sobre los residuos del modelo.
</p>

In [ ]:
import pylab
import scipy.stats as stats
%matplotlib inline
pylab.figure(figsize=(15,10))
errors = yhat_train - ytrain
stats.probplot(errors,dist="norm",plot=pylab)
pylab.show()
print("error promedio entre hipótesis de normalidad y datos de entrenamiento = ", abs(errors.mean()))

<p  style="text-align: justify;">
    Se puede apreciar en el gráfico, que hay una gran cantidad de datos que se aproximan a la hipótesis de normalidad, por lo que sugiere que los datos de entrenamientos son buenos para hacer estimaciones. Esto se puede corroborar al calcular la media de los errores de entrenamiento, la cual es de $0.00012944196138753425$.
</p>

#  2 Selección de Atributos


<p  style="text-align: justify;">
    **(a)** Construya una función que implemente Forward Step-wise Selection (FSS). Es decir, partiendo con un
modelo sin predictores (variables), agregue un predictor a la vez, re-ajustando el modelo de regresión
en cada paso. Para seleccionar localmente una variable, proponga/implemente un criterio distinto al
utilizado en el código de ejemplo. Construya un gráfico que muestre el error de entrenamiento y el error
de pruebas como función del número de variables en el modelo. Ordene el eje x de menor a mayor.

</p>

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
plt.figure(figsize=(15,10))


def fss(x, y, names_x, test_data, k=10000):
    p = x.shape[1] - 1
    k = min(p, k)
    names_x = np.array(names_x)
    remaining = list(range(0, p))
    selected = [p]
    training_error = []
    test_error = []
    current_score = 0.0
    best_new_score = 0.0
    while remaining and len(selected) <= k:
        score_candidates = []
        for candidate in remaining:
            model = lm.LinearRegression(fit_intercept=False)
            indexes = selected + [candidate]
            X_train = x[:, indexes]
            model.fit(X_train, y)
            predicted_value_train = model.predict(X_train)
            residual_value_train = predicted_value_train - y
            X_test, y_test = test_data
            X_test = X_test[:, indexes]
            predicted_value_test = model.predict(X_test)
            residual_value_test = predicted_value_test - y_test
            mse_train = np.mean(np.power(residual_value_train, 2))
            mse_test = np.mean(np.power(residual_value_test, 2))
            var = (mse_train * X_train.shape[0]) / (X_train.shape[0] - X_train.shape[1] - 1)
            diag_values = np.diag(np.linalg.pinv(np.dot(X_train.T, X_train)))
            z_score = np.divide(model.coef_, np.sqrt(np.multiply(var, diag_values)))
            z_score_candidate = z_score[-1]
            score_candidates.append((z_score_candidate, mse_train, mse_test, candidate))
        score_candidates.sort()
        z_score, best_new_score_, mse_test_, best_candidate = score_candidates.pop()
        if best_candidate >= len(names_x):
            break
        remaining.remove(best_candidate)
        selected.append(best_candidate)
        training_error.append((len(selected), best_new_score_))
        test_error.append((len(selected), mse_test_))
        #print("selected= %s..." % names_x[best_candidate])
        #print("totalvars=%d, mse = %f" % (len(indexes), best_new_score_))
    return selected, training_error, test_error

X = df_scaled.iloc[:,1:]
names_regressors = X.columns[:-1]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['price']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
Xm_train = Xtrain.as_matrix()
ym_train = ytrain.as_matrix()
Xm_test = Xtest.as_matrix()
ym_test = ytest.as_matrix()
selected, points_training, points_test = fss(Xm_train, ym_train, names_regressors, (Xm_test, ym_test))
x_test= []
y_test= []
for x in points_test:
    x_test.append(x[0]-1)
    y_test.append(x[1])
x_train=[]
y_train=[]
for x in points_training:
    x_train.append(x[0]-1)
    y_train.append(x[1])
plt.plot(x_train, y_train, label='Training Set')
plt.plot(x_test, y_test, label='Testing Set')
plt.xlim(min(x_train), max(x_train))
plt.legend()
plt.xlabel('Cantidad de predictores')
plt.ylabel('Error de predicción')
plt.title('Train error vs Test error con FSS')
plt.legend(loc=2)
plt.show()

<p  style="text-align: justify;">
    Se utiliza primero que todo <b><i>FSS</i></b> para ir escogiendo los atributos que generan el menor error considerando los <b><i>Z-Score</i></b> como método de tolerancia.<br>
    Del gráfico se ve que ambos sets, tanto el de entrenamiento como el de testeo tienen muy poca variación, esto comprueba o reafirma la calidad de los datos de entrenamiento, ya que como se vio en el punto <b><i>h)</i></b>, el error entre los datos de entrenamiento y de testeo es muy pequeño, por lo que en este caso en específico, no ocurre un overfitting al considerar todas las variables en los datos de entrenamiento. <br>
    En el gráfico se plasma que el error de predicción disminuye conforme se agregan atributos, este comportamiento es consecuente con todos los resultados anteriormente obtenidos que califican a estos datos de entrenamientos como una buena elección para testearlos, ya que la diferencia de error entre los datos de entrenamiento y los de testeo es del orden de $10^{-2}$.
</p>

<p  style="text-align: justify;">

</p>

#  3 Regularización

<p  style="text-align: justify;">
    **(a)** Ajuste un modelo lineal utilizando "Ridge Regression", es decir, regularizando con la norma $l_2$. Utilice
valores del parámetro de regularización $\lambda$ en el rango $[10^7, 10^1]$, variando si estima conveniente.
Construya un gráfico que muestre los coeficientes obtenidos como función del parámetro de regularización. Describa lo que observa. (WARNING: Note que la línea 3 y el primer argumento en la línea 9
son críticos).

</p>

In [ ]:
from sklearn.linear_model import Ridge
import matplotlib.pylab as plt
X2 = X.drop('intercept', axis=1,inplace=False)
Xtrain = X2[istrain]
ytrain = y[istrain]
names_regressors = X2.columns
alphas_ = np.logspace(7,1,base=10)
coefs = []
model = Ridge(fit_intercept=True,solver='svd')
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
plt.figure(figsize=(18,9))
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
    plt.plot(alphas_, y_arr, label=label)
    plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.title('Regularization Path RIDGE')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

<p  style="text-align: justify;">
    **(b)** Ajuste un modelo lineal utilizando el método "Lasso", es decir, regularizando con la norma $l_1$. Utilice
valores del parámetro de regularización $\lambda$ en el rango $[10^0, 10^-3]$. Para obtener el código, modifique
las líneas 7 y 9 del ejemplo anterior. Construya un gráfico que muestre los coeficientes obtenidos
como función del parámetro de regularización. Describa lo que observa. ¿Es más efectivo Lasso para
seleccionar atributos?.

</p>

In [ ]:
from sklearn.linear_model import Lasso
import matplotlib.pylab as plt
X2 = X.drop('intercept', axis=1,inplace=False)
Xtrain = X2[istrain]
ytrain = y[istrain]
names_regressors = X2.columns
alphas_ = np.logspace(0,-3,base=10)
coefs = []
model = Lasso(fit_intercept=True)
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
plt.figure(figsize=(18,9))
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
    plt.plot(alphas_, y_arr, label=label)
    plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.title('Regularization Path Lasso')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

<p  style="text-align: justify;">
    **(c)** Escogiendo uno de los dos métodos regularizadores anteriores, especificando el porqué, construya un
gráfico que muestre el error de entrenamiento y el error de pruebas como función del parámetro de
regularizacion. Discuta lo que observa.
</p>

In [ ]:
Xtest = X2[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
alphas_ = np.logspace(7,1,base=10) #np.logspace(0,-3,base=10)
coefs = []
model = Ridge(fit_intercept=True,solver='svd')  #Lasso(fit_intercept=True) 
mse_test = []
mse_train = []
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    yhat_train = model.predict(Xtrain)
    yhat_test = model.predict(Xtest)
    mse_train.append(np.mean(np.power(yhat_train - ytrain, 2)))
    mse_test.append(np.mean(np.power(yhat_test - ytest, 2)))
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error ridge')
ax.plot(alphas_,mse_test,label='test error ridge')
plt.legend(loc=1)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

<p  style="text-align: justify;">
    **(d)** Estime el valor del parámetro de regularización en **alguno** de los modelos anteriores haciendo uso de
la técnica validación cruzada.

</p>

In [ ]:
def MSE(y,yhat): return np.mean(np.power(y-yhat,2))
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
best_cv_mse = float("inf")
model = Ridge(fit_intercept=True,solver='svd')
alphas_ = np.logspace(7,1,base=10)
for a in alphas_:
    model.set_params(alpha=a)
    mse_list_k10 = [MSE(model.fit(Xm[train], ym[train]).predict(Xm[val]), ym[val]) for train, val in kf.split(Xm)]
    if np.mean(mse_list_k10) < best_cv_mse:
        best_cv_mse = np.mean(mse_list_k10)
        best_alpha = a
        print ("BEST PARAMETER=%f, MSE(CV)=%f"%(best_alpha,best_cv_mse))

#  4 Drift

<p  style="text-align: justify;">
    En esta sección se presentarán dos muestras del dataframe utilizado en la actividades anteriores, donde cada
una de estas tiene una propiedad distinta ya que son muestreadas en función del valor a predecir (logaritmo
del precio de la casa). Por una parte se tiene una pequeña muestra A, la cual es extraída directamente de
los datos con los que se trabaja (manteniendo la distribución de esta) y la muestra B, es generada con el
propósito de que en cada intervalo del rango de valores haya la misma cantidad de datos aproximadamente
(simulando una distribución uniforme). El objetivo es familiarizarse con el concepto de _Transfer Learning_.
</p>

<p  style="text-align: justify;">
    El Aprendizaje por Transferencia (en inglés, _“Transfer Learning”_) es un problema de investigación en el aprendizaje automático que se centra en almacenar el conocimiento adquirido al resolver un problema y aplicarlo a otro problema diferente pero relacionado.
</p>

In [ ]:
df_A = df_scaled.sample(1000,random_state=11)

frames = []
valor = df_scaled.price
length = 0.3
for z in np.arange(int(np.min(valor)),int(np.max(valor))+1,length):
    #un maximo de 100 datos por intervalo
    aux = df_scaled[(df_scaled.price >= z) & (df_scaled.price < z+length)].head(100)
    frames.append(aux)
    
df_B = pd.concat(frames).sample(1000,random_state=11) #crea el dataframe

<p  style="text-align: justify;">
    **(a)** Cree el conjunto de entrenamiento y otro de validación para trabajar cada muestra mediante la técnica
de _hold out validation_.

</p>

In [ ]:
X_A = df_A.iloc[:,1:].values
y_A = df_A.price
X_B = df_B.iloc[:,1:].values
y_B = df_B.price
from sklearn.model_selection import train_test_split

Xtrain_A,Xval_A,ytrain_A,yval_A = train_test_split(X_A, y_A, test_size=0.3, random_state=42)
Xtrain_B,Xval_B,ytrain_B,yval_B = train_test_split(X_B, y_B, test_size=0.3, random_state=42)



<p  style="text-align: justify;">
    **(b)** Evalúe los dos modelo de regresión lineal que se generan al entrenar con cada muestra. Mida el error
de cada modelo sobre ambos conjuntos de validación (A y B). Explique lo que observa.
</p>


In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score

LR_A = lm.LinearRegression(fit_intercept = False)
LR_B = lm.LinearRegression(fit_intercept = False)

LR_A.fit(Xtrain_A, ytrain_A)
LR_B.fit(Xtrain_B, ytrain_B)


print ("Regresion lineal\nCV: ",LR_A.score(Xtrain_A, ytrain_A),"\nTest: ",LR_A.score(Xval_A, yval_A))

print ("Regresion lineal\nCV: ",LR_B.score(Xtrain_B, ytrain_B),"\nTest: ",LR_A.score(Xval_B, yval_B))


#print(accuracy_score(yval_A, result_A))

#print(LR_A.score(Xval_A, yval_A))
#print(result_B.score(Xval_B, yval_B))







<p  style="text-align: justify;">
    **(c)** Si tuviera que elegir uno de los dos modelos anteriores para trabajar con data futura, ¿Cuál eligiría y
por qué?

</p>

<p  style="text-align: justify;">
    Respuesta
</p>

#  5 Detectar enfermedades cardíacas

<p  style="text-align: justify;">
    En el área de la salud, diagnosticar a una persona de una enfermedad de forma rápida y correcta puede llegar
a salvarle la vida. Los encargados de realizar estos diagnósticos, son médicos que, observando exámenes y
ciertos indicadores, pueden concluir qué enfermedad presenta el paciente. Si el medico se llegase a equivocar,
aparte de que el paciente pueda perder la vida, el medico podría ser demandado por negligencia arriesgando
a~nos de cárcel o pagar sumas de dinero considerable, es por estas razones que es importante no cometer
errores.
Pongámonos en el contexto de que usted es contratado para generar un modelo que prediga si es que un
paciente presenta una enfermedad cardiaca a partir de ciertos indicadores, tales como la edad, sexo, presión sanguínea, nivel de glicemia, etc. Los datos para trabajar junto a su documentación pueden ser descargados
ejecutando los siguientes comandos en un terminal (sistemas UNIX)
</p>

- wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat
- wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.doc

<p  style="text-align: justify;">
    Como ayuda se le indica que la variable de máximo ritmo cardíaco alcanzado (_maximum heart rate achieved_)
es un buen indicador de detección de enfermedades cardíacas. Por lo que el objetivo es predecir el comportamiento
de esta variable en función de las otras, y con esto detectar qué tan distante es el valor real al valor
predecido para así luego detectar los posibles _outliers_ (enfermos), que en sí corresponden a pacientes que
tienen un comportamiento anormal al resto.

</p>

<p  style="text-align: justify;">
   **(a)** Lea el archivo de datos, cárguelos en un dataframe o matriz, luego divida el dataframe en dos, un
dataframe de entrenamiento (70% Datos) y un dataframe de prueba (30% Datos).

</p>

In [ ]:
headers = ["age","sex","chest_pain","blood_p","serum","blood_s","electro","max_heart","angina"
           ,"oldpeak","slope","vessel","thal","normal"]


df1 = pd.read_table("heart.dat", header=None, names=headers, sep=" ", engine="python")
mx = df1["max_heart"]
df1.drop(["max_heart"],axis=1,inplace=True)


#print(df2)
#variable normal: Absence (1) or Presence (2) of Heart Disease
train, test, goal_train, goal_test = train_test_split(df1.loc[:,["age","sex","chest_pain","blood_p","serum","blood_s","electro","angina"
           ,"oldpeak","slope","vessel","thal"]], 
                                                      mx, 
                                                      test_size=0.3, random_state=0)
print(train)
print(goal_train)


df1.describe()


<p  style="text-align: justify;">
   **(b)** Realice una regresión lineal y defina usted una frontera de decisión (umbral) para poder determinar si
es que estamos en presencia o no de una enfermedad cardíaca. Mida su desempeño con ambos conjuntos
de datos.

</p>

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

LR = lm.LinearRegression()
LR.fit(train, goal_train)

result_train = np.rint(LR.predict(train))
result_test = np.rint(LR.predict(test))
print(result_train)

print ("Test Score: {:8.8f}".format(accuracy_score(goal_test, result_test)))
print ("Train Score: {:8.8f}".format(accuracy_score(goal_train, result_train)))




<div id='obj' />
# Referencias

<ul style="text-align: justify;">
    <li>https://www.kaggle.com/harlfoxem/housesalesprediction/discussion/23194</li>
    <li>https://rstudio-pubs-static.s3.amazonaws.com/155304_cc51f448116744069664b35e7762999f.html</li>
    <li>http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html</li>
    <li>https://fisicamatematic.wordpress.com/2011/11/04/minimos-cuadrados</li>
    <li></li>
    <li></li>
</ul>

<div id='obj' />

